<a href="https://colab.research.google.com/github/heber-augusto/sus-kpis-analysis/blob/main/sia/notebook_cancer_mama_kpis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração do ambiente e carregamento do catálogo

**Lembre-se de baixar o arquivo json que garante acesso ao google storage**
 - monitor-rosa-escrita.json: acesso para criação de tabelas;
 - monitor-rosa-leitura.json: acesso para analises e consultas.

**Se o seu usuário possui acesso ao drive compartilhado, remova os comentários a seguir**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Escolha do ambiente
São disponibilizadas duas opções de execução (dependendo do arquivo de credencial utilizado)
 - gcp-leitura.json: disponível para analises e usuários que não possuem permissão de escrita;
 - gcp-escrita.json: disponível para atualização de bases (testes de desenvolvimento ou ambiente de produção).

In [ ]:
import os
if os.path.isfile('/content/monitor-rosa-leitura.json'):
    datalake_mode = 'leitura'
    %env SERVICE_ACCOUNT_USER=acesso-leitura@monitor-rosa.iam.gserviceaccount.com
    %env SERVICE_ACCOUNT_JSON=/content/monitor-rosa-leitura.json
elif os.path.isfile('/content/monitor-rosa-escrita.json'):
    datalake_mode = 'escrita'
    %env SERVICE_ACCOUNT_USER=acesso-escrita@monitor-rosa.iam.gserviceaccount.com
    %env SERVICE_ACCOUNT_JSON=/content/monitor-rosa-escrita.json
else:
    assert(os.path.isdir('/content/drive/Shareddrives/monitor-rosa-gold') == True)
    datalake_mode = 'shared_drive'
    %env SERVICE_ACCOUNT_USER=''
    %env SERVICE_ACCOUNT_JSON=''
datalake_mode

env: SERVICE_ACCOUNT_USER=''
env: SERVICE_ACCOUNT_JSON=''


'shared_drive'

In [ ]:
!rm -r sus-kpis-analysis
!git clone https://github.com/heber-augusto/sus-kpis-analysis.git

rm: cannot remove 'sus-kpis-analysis': No such file or directory
Cloning into 'sus-kpis-analysis'...
remote: Enumerating objects: 1731, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 1731 (delta 105), reused 204 (delta 83), pack-reused 1464 (from 1)
Receiving objects: 100% (1731/1731), 4.71 MiB | 22.43 MiB/s, done.
Resolving deltas: 100% (806/806), done.


## Instalação de libs Python, inicialização de variáveis de ambiente e configuração/instalação do Spark

In [ ]:
!pip install -r /content/sus-kpis-analysis/sia/etls/requirements.txt

%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/content/spark-3.4.3-bin-hadoop3
%env SPARK_VERSION=3.4.3

!source /content/sus-kpis-analysis/sia/etls/bin/setup_spark_env.sh '/content/'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 68.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.3-py2.py3-none-any.whl size=311885495 sha256=ddd2b2e2b900324833a82a7e8b52b8369d2bdbe4641b3b9a4975d92b8c50a22a
  Stored in directory: /root/.cache/pip/wheels/ea/a9/64/3713eb2c5048c18bae2778b013e5fc74203f5c22d4640fb776
Successfully built pyspark
env: PYTHONHASHSEED=1234
env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
env: SPARK_HOME=/content/spark-3.4.3-bin-hadoop3
env: SPARK_VERSION=3.4.3
starting spark env setup 
installing and downloading packages
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat

In [ ]:
if datalake_mode != 'shared_drive':
    %env XDG_CONFIG_HOME=/content/datalake
    !source /content/sus-kpis-analysis/sia/etls/bin/install-google-drive-ocamlfuse.sh
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0ABIY-a4qrdY9Uk9PVA' 'monitor-rosa-bronze' $SERVICE_ACCOUNT_JSON '/content'
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0ALl0owLNr53oUk9PVA' 'monitor-rosa-silver' $SERVICE_ACCOUNT_JSON '/content'
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0AMHp9pBeLvZiUk9PVA' 'monitor-rosa-gold' $SERVICE_ACCOUNT_JSON '/content'


## Inicializa variáveis de acesso ao delta lake criado no google storage

- Local do arquivo de credencial do storage, diretorio do warehouse e path do spark:

> O arquivo json_file_name deve ser enviado para o ambiente e deve ser utilizado um com as devidas permissões (em caso de escrita)

> O caminho do warehouse pode ser alterado em caso de testes de escritas locais.

> O caminho do spark é setado pelo script de configuração

In [ ]:
import os

lake_prefix = "temp-output"

if datalake_mode in ('leitura','shared_drive',):
    warehouse_dir = f"/content/datalake/{lake_prefix}/"

if datalake_mode == 'escrita':
    warehouse_dir = f"/content/datalake/"

spark_path = os.getenv('SPARK_HOME')
spark_path

'/content/spark-3.4.3-bin-hadoop3'

## Inclusão da pasta do repositório no python path

Procedimento permite que funções e classes presentes no repositório sejam utilizadas

In [ ]:
import sys
sys.path.append('/content/sus-kpis-analysis')
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor',
 '/root/.ipython',
 '/content/sus-kpis-analysis']

## Importação de funções utilizadas pelo código

In [ ]:
from sia.etls.lib.catalog_loader import DeltaLakeDatabaseFsCreator, load_entire_catalog_fs, load_entire_catalog_fs_v2
from sia.etls.lib.table_utilities import vacuum_tables_from_database, table_exists
from sia.etls.lib.fs_spark_session import create_fs_spark_session
from sia.etls.lib.bronze_files_utilities import get_pending_files_from_bronze
from sia.etls.lib.delta_table_creators import ParquetToDelta

## Cria Sessão Spark conectada ao Delta Lake presente no Google Storage

In [ ]:
spark = create_fs_spark_session(
    warehouse_dir=warehouse_dir,
    spark_path=spark_path
)


## Refresh do catálogo para utilizar consultas

In [ ]:
zone_names = ['monitor-rosa-bronze', 'monitor-rosa-silver', 'monitor-rosa-gold']
if datalake_mode in ('leitura', 'escrita'):
    zone_paths = [f'/content/datalake/{zone_name}/databases' for zone_name in zone_names]
else:
    zone_paths = [f'/content/drive/Shareddrives/{zone_name}/databases' for zone_name in zone_names]


# carrega catalogo de banco de dados, na zona bronze
database_filter = None #['cnes_bronze.db',]
table_filter = ['sia_bronze.ar','sia_bronze.aq','cancer_data.cadastro_municipios', 'cancer_data.demografia_municipios']

for databases_path in zone_paths:
    load_entire_catalog_fs_v2(
        spark_session = spark,
        databases_path = databases_path,
        use_db_folder_path=(datalake_mode == 'escrita'),
        database_filter=database_filter,
        table_filter=table_filter
    )

['sia_bronze.db', 'cnes_bronze.db', 'sih_bronze.db', 'sim_bronze.db']
Banco de dados sia_bronze criado.
listando conteúdos do caminho /content/drive/Shareddrives/monitor-rosa-bronze/databases e database sia_bronze
prefix: /content/drive/Shareddrives/monitor-rosa-bronze/databases/sia_bronze.db/
table_list: ['ar', 'aq', 'pa', 'bi', 'am']
Tabela ar criada
Tabela ar criada com comando CREATE TABLE IF NOT EXISTS sia_bronze.ar USING delta LOCATION '/content/drive/Shareddrives/monitor-rosa-bronze/databases/sia_bronze.db/ar'
Tabela aq criada
Tabela aq criada com comando CREATE TABLE IF NOT EXISTS sia_bronze.aq USING delta LOCATION '/content/drive/Shareddrives/monitor-rosa-bronze/databases/sia_bronze.db/aq'
Recriação das tabelas concluída.
Banco de dados cnes_bronze criado.
listando conteúdos do caminho /content/drive/Shareddrives/monitor-rosa-bronze/databases e database cnes_bronze
prefix: /content/drive/Shareddrives/monitor-rosa-bronze/databases/cnes_bronze.db/
table_list: ['dc', 'lt', 'ep', 

# Exemplos de consultas

## Listagem de bancos e tabelas

In [ ]:
databases = spark.sql(f"SHOW DATABASES;")
databases.show()

+-----------+
|  namespace|
+-----------+
|cancer_data|
|cnes_bronze|
|    default|
| sia_bronze|
| sih_bronze|
| sim_bronze|
+-----------+



In [ ]:
for row in databases.collect():
    spark.sql(f"SHOW TABLES FROM {row['namespace']};").show(truncate=False)

+-----------+---------------------+-----------+
|namespace  |tableName            |isTemporary|
+-----------+---------------------+-----------+
|cancer_data|cadastro_municipios  |false      |
|cancer_data|demografia_municipios|false      |
+-----------+---------------------+-----------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+

+----------+---------+-----------+
|namespace |tableName|isTemporary|
+----------+---------+-----------+
|sia_bronze|aq       |false      |
|sia_bronze|ar       |false      |
+----------+---------+-----------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+

In [ ]:
def get_select_all_query (table_name, where_clause = ''):
    return f"""
    SELECT
        *
    FROM {table_name}
    {where_clause}
    """

def run_sql_query(sql_query):
    return spark.sql(sql_query)

In [ ]:
# filtro pelo cid
cid_filter = ['C500', 'C501', 'C502', 'C503', 'C504', 'C505', 'C506', 'C508', 'C509']

# dicionario de procedimentos
proc_id_dict = {
    '0201010569': 'BIOPSIA/EXERESE DE NÓDULO DE MAMA',
    '0201010585': 'PUNÇÃO ASPIRATIVA DE MAMA POR AGULHA FINA',
    '0201010607': 'PUNÇÃO DE MAMA POR AGULHA GROSSA',
    '0203010035': 'EXAME DE CITOLOGIA (EXCETO CERVICO-VAGINAL E DE MAMA)',
    '0203010043': 'EXAME CITOPATOLOGICO DE MAMA',
    '0203020065': 'EXAME ANATOMOPATOLOGICO DE MAMA - BIOPSIA',
    '0203020073': 'EXAME ANATOMOPATOLOGICO DE MAMA - PECA CIRURGICA',
    '0205020097': 'ULTRASSONOGRAFIA MAMARIA BILATERAL',
    '0208090037': 'CINTILOGRAFIA DE MAMA (BILATERAL)',
    '0204030030': 'MAMOGRAFIA',
    '0204030188': 'MAMOGRAFIA BILATERAL PARA RASTREAMENTO'
    }

proc_id_filter = [f"'{proc_id}'" for proc_id in list(proc_id_dict.keys())]
proc_id_filter = f"""({','.join(proc_id_filter)})"""
print(proc_id_filter)
cid_filter = [f"'{cid_id}'" for cid_id in cid_filter]
cid_filter = f"""({','.join(cid_filter)})"""
print(cid_filter)

('0201010569','0201010585','0201010607','0203010035','0203010043','0203020065','0203020073','0205020097','0208090037','0204030030','0204030188')
('C500','C501','C502','C503','C504','C505','C506','C508','C509')


# Carrega tabela SIA.AQ filtrando dados de câncer de mama

In [ ]:
sql_query = get_select_all_query(
    table_name = 'sia_bronze.aq'  ,
    where_clause = f"WHERE AP_CIDPRI IN {cid_filter} AND AP_MVM = '202405'"  ,
)

cancer_aq_filtered = run_sql_query(sql_query)

In [ ]:
cancer_aq_filtered\
      .repartition(1)\
      .write\
      .format("delta")\
      .mode("overwrite")\
      .saveAsTable("cancer_data.aq_filtered1")

# Carrega tabela SIA.AR filtrando dados de câncer de mama

In [ ]:
sql_query = get_select_all_query(
    table_name = 'sia_bronze.ar'  ,
    where_clause = f"WHERE AP_CIDPRI IN {cid_filter} AND AP_MVM = '202405'"  ,
)

cancer_ar_filtered = run_sql_query(sql_query)

In [ ]:
cancer_ar_filtered\
      .repartition(1)\
      .write\
      .format("delta")\
      .mode("overwrite")\
      .saveAsTable("cancer_data.ar_filtered1")

# Cria dados consolidados de pacientes e procedimentos (quimio e radioterapia)

In [ ]:

cancer_aq_res = spark.sql(f"""
SELECT
    AP_CMP as data,
    AP_CNSPCN as paciente,
    AQ_ESTADI as estadiamento,
    DOUBLE(AP_VL_AP)  as custo,
    INT(AP_OBITO) as obito,
    AP_MUNPCN as municipio
FROM cancer_data.aq_filtered1
""")

cancer_ar_res = spark.sql(f"""
SELECT
    AP_CMP as data,
    AP_CNSPCN as paciente,
    AR_ESTADI as estadiamento,
    DOUBLE(AP_VL_AP)  as custo,
    INT(AP_OBITO) as obito,
    AP_MUNPCN as municipio
FROM cancer_data.ar_filtered1
""")


df_union = cancer_aq_res.union(cancer_ar_res)


df_union.createOrReplaceTempView("cancer_ordered")


res_consolidado = spark.sql("""
SELECT
    paciente,
    FIRST(data) as data_primeiro_estadiamento,
    LAST(data) as data_ultimo_estadiamento,
    COUNT(1) as numero_procedimentos,
    FIRST(estadiamento) as primeiro_estadiamento,
    LAST(estadiamento) as ultimo_estadiamento,
    MAX (estadiamento) as maior_estadiamento,
    MIN (estadiamento) as menor_estadiamento,
    SUM(custo) as custo_total,
    MAX(obito) as indicacao_obito,
    FIRST(municipio) as primeiro_municipio,
    LAST(municipio) as ultimo_municipio
FROM (SELECT * FROM cancer_ordered ORDER BY paciente, data)
GROUP BY paciente
""")

In [ ]:
df_union\
  .repartition(1)\
  .write\
  .format("delta")\
  .mode("overwrite")\
  .saveAsTable(f"cancer_data.procedimentos")

res_consolidado\
  .repartition(1)\
  .write\
  .format("delta")\
  .mode("overwrite")\
  .saveAsTable(f"cancer_data.pacientes")

In [ ]:
procedimentos_e_pacientes = spark.sql(f"""
  select
      c.*,
      p.data_primeiro_estadiamento,
      p.data_ultimo_estadiamento,
      p.primeiro_estadiamento,
      p.maior_estadiamento,
      p.ultimo_estadiamento,
      p.custo_total,
      p.primeiro_municipio,
      p.ultimo_municipio,
      p.indicacao_obito
  from cancer_data.procedimentos as c
  FULL OUTER JOIN cancer_data.pacientes as p
  ON c.paciente = p.paciente
  """)

procedimentos_e_pacientes\
  .repartition(1)\
  .write\
  .format("delta")\
  .mode("overwrite")\
  .saveAsTable(f"cancer_data.procedimentos_e_pacientes")